<a href="https://colab.research.google.com/github/thxsxth/RLMimic/blob/master/Sepsis3_Cohort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd 'drive/My Drive/sepsis3-cohort'

/content/drive/My Drive/sepsis3-cohort


In [0]:
import numpy as np
import pandas as pd


### The following 'sepsis3-cohort.csv' file was generated by  GCP Big Query and modification of code https://github.com/alistairewj/sepsis3-mimic

We will manually changing the suspicion of infection column from, 'sus_poe_table.csv' which as the the pivoted values. Big Query suspected_infection_time doesn't make sense with respect to the the analysis of the paper/github

In [0]:
date_cols = ['intime', 'outtime','suspected_infection_time_poe']
df=pd.read_csv('sepsis3-cohort.csv',parse_dates=date_cols)

In [30]:
df.head(5)

,icustay_id,hadm_id,excluded,intime,outtime,dbsource,suspected_infection_time_poe,suspected_infection_time_poe_days,specimen_poe,positiveculture_poe,antibiotic_time_poe,blood_culture_time,blood_culture_positive,age,gender,is_male,ethnicity,race_white,race_black,race_hispanic,race_other,Height,Weight,bmi,first_service,HOSPITAL_EXPIRE_FLAG,THIRTYDAY_EXPIRE_FLAG,icu_los,hosp_los,sepsis_angus,sepsis_martin,sepsis_explicit,septic_shock_explicit,severe_sepsis_explicit,sepsis_nqf,sepsis_cdc,sepsis_cdc_simple,vent,sofa,lods,sirs,qsofa,qsofa_sysbp_score,qsofa_gcs_score,qsofa_resprate_score
0,200001,152234,1,2181-11-25 19:06:12,2181-11-28 20:59:25,metavision,NaT,NaN,NaN,NaN,2181-11-23 00:00:00,2181-11-24 00:12:00,1,61.0,F,0,ASIAN - ASIAN INDIAN,0,0,0,1,170.180000,61.0,21.062640,MED,0,0,3.0786,16,0,0,0,0,0,0,0,0,0,6,7,3,2,1.0,NaN,1.0
1,200003,163557,1,2199-08-02 19:50:04,2199-08-08 17:09:18,carevue,NaT,NaN,NaN,NaN,2199-08-02 00:00:00,2199-08-02 21:02:00,1,48.0,M,1,WHITE,1,0,0,0,177.800000,77.0,24.357192,SURG,0,0,5.8884,20,1,1,0,0,0,1,0,0,0,4,4,4,2,1.0,0.0,1.0
2,200006,189514,1,2159-09-03 11:28:14,2159-09-04 19:08:10,carevue,NaT,NaN,NaN,NaN,NaN,2159-09-03 09:00:00,1,54.0,M,1,OTHER,0,0,0,1,168.002857,82.4,29.194018,MED,0,0,1.3194,2,0,0,0,0,0,0,0,0,0,1,2,2,2,1.0,0.0,1.0
3,200007,129310,1,2109-02-17 10:03:37,2109-02-18 17:03:12,carevue,NaT,NaN,NaN,NaN,NaN,NaN,0,44.0,M,1,WHITE,1,0,0,0,177.800000,126.0,39.857223,CMED,0,0,1.2914,3,0,0,0,0,0,0,0,0,0,1,2,2,2,1.0,0.0,1.0
4,200009,129607,1,2189-11-30 10:34:32,2189-12-02 14:17:37,carevue,NaT,NaN,NaN,NaN,2189-12-01 00:00:00,NaN,0,47.0,F,0,WHITE,1,0,0,0,161.290000,87.2,33.519814,CSURG,0,0,2.1549,6,0,0,0,0,0,0,0,0,0,3,4,4,2,1.0,0.0,1.0


In [0]:
sus=pd.read_csv('sus_poe_table.csv',parse_dates=['suspected_infection_time'])

In [14]:
sus['suspected_infection_time'].describe()

count                 1247140
unique                 185457
top       2147-06-28 12:00:00
freq                      384
first     2100-06-07 18:30:00
last      2210-08-22 00:00:00
Name: suspected_infection_time, dtype: object

In [0]:
# a=sus.groupby(['icustay_id'])['suspected_infection_time'].min()

In [0]:
df['suspected_infection_time_poe']=sus.groupby(['icustay_id'])['suspected_infection_time'].min().values

In [95]:
df.head()

,icustay_id,hadm_id,excluded,intime,outtime,dbsource,suspected_infection_time_poe,suspected_infection_time_poe_days,specimen_poe,positiveculture_poe,antibiotic_time_poe,blood_culture_time,blood_culture_positive,age,gender,is_male,ethnicity,race_white,race_black,race_hispanic,race_other,Height,Weight,bmi,first_service,HOSPITAL_EXPIRE_FLAG,THIRTYDAY_EXPIRE_FLAG,icu_los,hosp_los,sepsis_angus,sepsis_martin,sepsis_explicit,septic_shock_explicit,severe_sepsis_explicit,sepsis_nqf,sepsis_cdc,sepsis_cdc_simple,vent,sofa,lods,sirs,qsofa,qsofa_sysbp_score,qsofa_gcs_score,qsofa_resprate_score
0,200001,152234,1,2181-11-25 19:06:12,2181-11-28 20:59:25,metavision,2181-11-18,NaN,NaN,NaN,2181-11-23 00:00:00,2181-11-24 00:12:00,1,61.0,F,0,ASIAN - ASIAN INDIAN,0,0,0,1,170.180000,61.0,21.062640,MED,0,0,3.0786,16,0,0,0,0,0,0,0,0,0,6,7,3,2,1.0,NaN,1.0
1,200003,163557,1,2199-08-02 19:50:04,2199-08-08 17:09:18,carevue,2199-08-02,NaN,NaN,NaN,2199-08-02 00:00:00,2199-08-02 21:02:00,1,48.0,M,1,WHITE,1,0,0,0,177.800000,77.0,24.357192,SURG,0,0,5.8884,20,1,1,0,0,0,1,0,0,0,4,4,4,2,1.0,0.0,1.0
2,200006,189514,1,2159-09-03 11:28:14,2159-09-04 19:08:10,carevue,NaT,NaN,NaN,NaN,NaN,2159-09-03 09:00:00,1,54.0,M,1,OTHER,0,0,0,1,168.002857,82.4,29.194018,MED,0,0,1.3194,2,0,0,0,0,0,0,0,0,0,1,2,2,2,1.0,0.0,1.0
3,200007,129310,1,2109-02-17 10:03:37,2109-02-18 17:03:12,carevue,NaT,NaN,NaN,NaN,NaN,NaN,0,44.0,M,1,WHITE,1,0,0,0,177.800000,126.0,39.857223,CMED,0,0,1.2914,3,0,0,0,0,0,0,0,0,0,1,2,2,2,1.0,0.0,1.0
4,200009,129607,1,2189-11-30 10:34:32,2189-12-02 14:17:37,carevue,2189-11-30,NaN,NaN,NaN,2189-12-01 00:00:00,NaN,0,47.0,F,0,WHITE,1,0,0,0,161.290000,87.2,33.519814,CSURG,0,0,2.1549,6,0,0,0,0,0,0,0,0,0,3,4,4,2,1.0,0.0,1.0


##### Let's verify the groupby method worked as expected

In [0]:
sus_dict={}
for id in sus[sus['suspected_infection_time'].notnull()]['icustay_id'].unique():
     temp=sus[sus['icustay_id']==id]
     sus_dict[id]=temp['suspected_infection_time'].min()

In [29]:
(set(df['suspected_infection_time_poe'])-(set(sus_dict.values())))

{NaT}

In [32]:
set(sus_dict.values())-set(df['suspected_infection_time_poe'])

set()

##### Worked as expected so Moving on

In [0]:
# df['suspected_infection_time_poe_days']=pd.Timedelta(pd.offsets.Day(df['suspected_infection_time_poe'].values-df['intime'].values))
# df.assign(suspected_infection_time_poe_days=lambda x:pd.Timedelta(pd.offsets.Day(x['suspected_infection_time_poe']-x['intime'])))

In [0]:
# pd.to_timedelta(df['suspected_infection_time_poe']-df['intime'])


In [0]:
# df.head(20)

One last comparison
Below cell would compare suspected_infection_times in BQ's already generated table to ours and it matches

In [0]:
df_bq=pd.read_csv('Sepsis3_bq.csv',parse_dates=['suspected_infection_time_poe'])
temp_df=df_bq[df_bq['suspected_infection_time_poe'].notna()][['icustay_id','suspected_infection_time_poe']]
print(df[df_bq['suspected_infection_time_poe'].notna()][['icustay_id','suspected_infection_time_poe']].head(50))
print(temp_df.head(50))


In [0]:
temp_df=df_bq[df_bq['suspected_infection_time_poe'].notna()][['icustay_id','suspected_infection_time_poe']]

In [0]:
# pat1=pd.read_csv('200001icustay.csv')

In [0]:
## pat1=pd.read_csv('200001icustay.csv')
# df[df['icustay_id'].isin(pat1.icustay_id)]

In [0]:
#  len(pat1.icustay_id.values)

### Working on the Cohort NO EXCLUSIONS

In [28]:
df.count()

icustay_id                           61532
hadm_id                              61532
excluded                             61532
intime                               61532
outtime                              61522
dbsource                             61532
suspected_infection_time_poe         41057
suspected_infection_time_poe_days     9754
specimen_poe                          9754
positiveculture_poe                   9754
antibiotic_time_poe                  40972
blood_culture_time                   46140
blood_culture_positive               61532
age                                  61532
gender                               61532
is_male                              61532
ethnicity                            61532
race_white                           61532
race_black                           61532
race_hispanic                        61532
race_other                           61532
Height                               35040
Weight                               47400
bmi        

In [0]:
from collections import OrderedDict
df['composite_outcome'] = ( (df['HOSPITAL_EXPIRE_FLAG']==1) | (df['icu_los']>=3) ).astype(int)

labels = OrderedDict([['suspicion_poe', 'BC + ABX (Prescribed)']])

# add some other useful variables
df['blood culture'] = (~df['blood_culture_time'].isnull())
df['suspicion_poe'] = (~df['suspected_infection_time_poe'].isnull())

df['abx_poe'] = (~df['antibiotic_time_poe'].isnull())

df['sepsis-3'] = ((df['suspicion_poe']==1) & (df['sofa']>=2)).astype(int)
df['sofa>=2'] = (df['sofa']>=2).astype(int)


In [0]:
for c in ['intime','outtime',
          'suspected_infection_time_mv','suspected_infection_time',
          'suspected_infection_time_poe', 'blood_culture_time']:
    if c in df.columns:
        if df[c].dtype == 'object':
            df[c] = pd.to_datetime(df[c])

# list of the sepsis definitions
sepsis_list = ['sepsis_angus','sepsis_martin', 'sepsis_explicit',
               'sepsis_cdc','sepsis_nqf',
               'sepsis-3']

In [35]:
def print_stats(df): 
    print('{:5g} patients.'.format(df.shape[0]))

    print('{:5g} ({:2.0f}%) suspected of infection.'.format(
        df['suspicion_poe'].sum(), df['suspicion_poe'].sum()*100.0/df.shape[0]))

    print('{:5g} ({:2.2f}%) have SOFA >= 2.'.format(
      df['sofa>=2'].sum(),100.0*df['sofa>=2'].mean()))

    print('{:5g} ({:2.2f}%) have Sepsis-3 criteria (intersection of above two).'.format(
      df['sepsis-3'].sum(),100.0*df['sepsis-3'].mean()))

    print('{:5g} ({:2.2f}%) have suspicion and SOFA < 2.'.format(
        ((df['sofa>=2']==0) & (df['suspicion_poe']==1)).sum(),
        ((df['sofa>=2']==0) & (df['suspicion_poe']==1)).sum()*100.0/df.shape[0]))

    for c in sepsis_list:
        print('{:5g} ({:3.2f}%) - {}'.format(
        df[c].sum(), df[c].sum()*100.0/df.shape[0], c))

    print('{:5g} ({:2.2f}%) have Sepsis-3 criteria but not Angus.'.format(
        ((df['sepsis_angus']==0) & (df['sepsis-3']==1)).sum(),
        ((df['sepsis_angus']==0) & (df['sepsis-3']==1)).sum()*100.0/df.shape[0]))

print_stats(df)

61532 patients.
41057 (67%) suspected of infection.
45668 (74.22%) have SOFA >= 2.
33666 (54.71%) have Sepsis-3 criteria (intersection of above two).
 7391 (12.01%) have suspicion and SOFA < 2.
17420 (28.31%) - sepsis_angus
 9028 (14.67%) - sepsis_martin
 4734 (7.69%) - sepsis_explicit
 4151 (6.75%) - sepsis_cdc
 6329 (10.29%) - sepsis_nqf
33666 (54.71%) - sepsis-3
19347 (31.44%) have Sepsis-3 criteria but not Angus.


In [44]:
print_stats(df[df['excluded']==0])

12965 patients.
 8763 (68%) suspected of infection.
 9801 (75.60%) have SOFA >= 2.
 7091 (54.69%) have Sepsis-3 criteria (intersection of above two).
 1672 (12.90%) have suspicion and SOFA < 2.
 4049 (31.23%) - sepsis_angus
 2010 (15.50%) - sepsis_martin
 1202 (9.27%) - sepsis_explicit
 1404 (10.83%) - sepsis_cdc
 1474 (11.37%) - sepsis_nqf
 7091 (54.69%) - sepsis-3
 3587 (27.67%) have Sepsis-3 criteria but not Angus.


In [0]:
sepsis3_df=df[df['sepsis-3']==1]

In [0]:
sepsis3_df.to_csv('cleaned_sepsis3_cohort.csv')

In [0]:
sepsis3_df=sepsis3_df[sepsis3_df['excluded']==0]

Statistics with exclusions

In [43]:
print_stats(sepsis3_df)

 7091 patients.
 7091 (100%) suspected of infection.
 7091 (100.00%) have SOFA >= 2.
 7091 (100.00%) have Sepsis-3 criteria (intersection of above two).
    0 (0.00%) have suspicion and SOFA < 2.
 3504 (49.41%) - sepsis_angus
 1828 (25.78%) - sepsis_martin
 1143 (16.12%) - sepsis_explicit
 1241 (17.50%) - sepsis_cdc
 1399 (19.73%) - sepsis_nqf
 7091 (100.00%) - sepsis-3
 3587 (50.59%) have Sepsis-3 criteria but not Angus.
